In [1]:
import pandas as pd
import gzip
import json
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances,cosine_similarity
from sklearn.metrics import mean_squared_error
import scipy.sparse as sp
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
import torch

from spotlight.interactions import Interactions
from spotlight.sequence.implicit import ImplicitSequenceModel
from spotlight.sequence.representations import CNNNet
from spotlight.evaluation import sequence_mrr_score

import torch
from spotlight.factorization.explicit import ExplicitFactorizationModel
from spotlight.cross_validation import random_train_test_split
from spotlight.interactions import Interactions
from spotlight.sequence.implicit import ImplicitSequenceModel
from spotlight.sequence.representations import CNNNet
from spotlight.evaluation import sequence_mrr_score
from spotlight.losses import regression_loss



In [2]:
amazon = pd.read_csv("feature_matrix_time.csv")
amazon.drop('Unnamed: 0',axis = 1, inplace =True)
label_u,level_u = pd.factorize(amazon['reviewerID'])
amazon['reviewerID'] = label_u+1
label_i,level_i = pd.factorize(amazon['asin'])
amazon['asin'] = label_i+1

In [3]:
amazon_interaction = Interactions(np.array(amazon['reviewerID']), 
                                  np.array(amazon['asin']),
                                  ratings = np.array(amazon['overall']), 
                                  timestamps=np.array(amazon['timestamps']))
train, test = random_train_test_split(amazon_interaction, random_state=np.random.RandomState(42))
train_seq = train.to_sequence()
test_seq = test.to_sequence()


In [ ]:
net = CNNNet(train_seq.num_items,
             embedding_dim=64,
             kernel_width=7,
             dilation=[1, 1, 1, 1, 1, 1, 1, 1, 1],
             num_layers=9,
             nonlinearity='tanh',
             residual_connections=True)
model = ImplicitSequenceModel(loss='hinge',
                              representation=net,
                              batch_size=16,
                              learning_rate=0.1,
                              l2=1e-3,
                              n_iter=10,
                              use_cuda=torch.cuda.is_available()
                             )

